This Notebook is the implementation of fine-tuning Mistral LLM on Financial Phrasebank Dataset for Sentiment Analysis task. Evaluation was performed on test dataset and results have been reported

### Install Libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.34.0 trl==0.4.7
!pip install huggingface_hub
!pip install -q datasets


import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
import pandas as pd
import random
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 4.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

### Load the data

In [4]:
### set display width as maximum for pandas dataframe to show the full data in the column
pd.set_option('display.max_colwidth', -1)

### load the dataset from Hugging Face hub, sentence_allagree version of the dataset will be used as mentioned in thesis document
dataset = load_dataset("financial_phrasebank",'sentences_allagree',split='train')


### Data is split into 75% Train and 25% Test
### The Target sentiment Label is picked in such a way (Stratified) that the proportion of sentiments is maintained in both Test and Train datasets

ds_split = dataset.train_test_split(test_size=0.25, stratify_by_column="label")

### Get the Train and Test Dataset from Hugging Face Dataset Dict object
ds_train = ds_split["train"]
ds_test = ds_split["test"]

train_dataset_dict = ds_train.train_test_split(test_size=0.1, stratify_by_column="label")
train_df = train_dataset_dict['train']
eval_df = train_dataset_dict['test']

### Convert Dataset into Pandas for Data preparation Steps
ds_train_pd = train_df.to_pandas()
ds_eval_pd = eval_df.to_pandas()
ds_test_pd = ds_test.to_pandas()


ds_train_pd['label'] = ds_train_pd['label'].map({0:'Negative',1:'Neutral',2:'Positive'})
ds_eval_pd['label'] = ds_eval_pd['label'].map({0:'Negative',1:'Neutral',2:'Positive'})


### List of Instructions
instructions = ['This statement has been extracted from a financial news article. Please provide your answer as either positive, negative or neutral.',
'The following financial news sentence has been provided as input. Determine if the sentiment is positive, negative, or neutral?',
'What is the sentiment of sentence extracted from financial news report? The provided options are positive, negative, or neutral.',
'Based on the text extracted from the financial news report, determine whether the sentiment is positive, negative, or neutral.',
'The sentiment of the text extracted from a financial news source needs to be evaluated. The sentiment could be positive, negative, or neutral.',
'This statement has been extracted from a financial news piece and its sentiment needs to be determined. Indicate whether it is positive, negative, or neutral.'
]
### create a list of 1698 instructions by randomly selecting instructions from above list and add it to train dataframe to create instructions column
ins_list = []
for i in range(len(ds_train_pd)):
  instr = random.choice(instructions)
  ins_list.append(instr)
ds_train_pd['instructions']  = ins_list


ins_list_eval = []
for i in range(len(ds_eval_pd)):
  instr_eval = random.choice(instructions)
  ins_list_eval.append(instr_eval)
ds_eval_pd['instructions']  = ins_list_eval


question_str = 'Determine the sentiment of text given as Input Text'
ds_train_pd['final_prompt'] = f'###Instruction:' + ds_train_pd['instructions'] + f'\n\n' + f'###Input Text:' + ds_train_pd['sentence'] + f'\n\n' + f'###Question:' + question_str + f'\n\n' + f'###Answer:' + ds_train_pd['label']


question_str = 'Determine the sentiment of text given as Input Text'
ds_eval_pd['final_prompt'] = f'###Instruction:' + ds_eval_pd['instructions'] + f'\n\n' + f'###Input Text:' + ds_eval_pd['sentence'] + f'\n\n' + f'###Question:' + question_str + f'\n\n' + f'###Answer:' + ds_eval_pd['label']

train_df = Dataset.from_pandas(ds_train_pd)
eval_df = Dataset.from_pandas(ds_eval_pd)



<ipython-input-4-00397069bbd1>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

### Fine-Tuning Framework Parameters

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"



# Fine-tuned model name
new_model = "mistral-7b-fpb-rank-64-vj_v1.1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 ### Rank Parameter

# Alpha parameter for LoRA scaling
lora_alpha = 128

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2000

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

device_map = {"": 0}

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Perform Fine-tuning

In [7]:
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load Mistral tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj"]
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_df,
    eval_dataset=eval_df,
    peft_config=peft_config,
    dataset_text_field="final_prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1528 [00:00<?, ? examples/s]

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.746500,0.757557


In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

In [8]:


# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-v0.1"



# Fine-tuned model name
new_model = "mistral-7b-fpb-rank-64-vj_v1.1"

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Save and Push fine-tuned model to hugging face hub

In [9]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/vijayshankar245/mistral-7b-fpb-rank-64-vj_v1.1/commit/22258d410b60273aaffeb5bdf96237392e9b887f', commit_message='Upload tokenizer', commit_description='', oid='22258d410b60273aaffeb5bdf96237392e9b887f', pr_url=None, pr_revision=None, pr_num=None)

#### Evaluation

### Prompt template for evaluation

In [10]:
question_str = 'Determine the sentiment of text given as Input Text'
ds_test_pd['final_prompt'] = f'###Input Text:' + ds_test_pd['sentence'] + f'\n\n' + f'###Question:' + question_str
test_df = Dataset.from_pandas(ds_test_pd)

### Load teh model for evaluation

In [11]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
model = "vijayshankar245/mistral-7b-fpb-rank-64-vj_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# Run text generation pipeline with our next model

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,return_full_text=False,temperature=0.6,top_p =0.9,max_new_tokens = 10)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Get the predictions

In [12]:
import time
start_time = time.time()
from transformers.pipelines.pt_utils import KeyDataset
output_list = []
j=0
for i in pipe(KeyDataset(test_df, "final_prompt")):
  j=j+1
  print(j,':',i[0]['generated_text'])
  output_list.append(i[0]['generated_text'])
print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


1 : 

###Answer:Positive


2 : 

###Answer:Neutral

3 : 

###Answer:Neutral

4 : 

###Answer:Neutral

5 : 

###Answer:Positive


6 : 

###Answer:Negative


7 : 

###Answer:Positive


8 : 

###Answer:Negative


9 : 

###Answer:Positive


10 : 

###Answer:Neutral

11 : 

###Answer:Neutral

12 : 

###Answer:Positive


13 : 

###Answer:Neutral

14 : 

###Answer:Neutral

15 : 

###Answer:Positive


16 : 

###Answer:Neutral

17 : 

###Answer:Neutral

18 : 

###Answer:Negative


19 : 

###Answer:Neutral

20 : 

###Answer:Neutral

21 : 

###Answer:Neutral

22 : 

###Answer:Neutral

23 : 

###Answer:Neutral

24 : 

###Answer:Neutral

25 : 

###Answer:Neutral

26 : 

###Answer:Negative


27 : 

###Answer:Neutral

28 : 

###Answer:Positive


29 : 

###Answer:Neutral

30 : 

###Answer:Positive


31 : 

###Answer:Negative


32 : 

###Answer:Neutral

33 : 

###Answer:Negative


34 : 

###Answer:Positive


35 : 

###Answer:Negative


36 : 

###Answer:Neutral

37 : 

###Answer:Positive


38 : 

###Ans

In [13]:
len(output_list)

566

### Post Processing

In [14]:
op_list = []
for txt in output_list:
  int1 = txt.replace(f"\n\n###Answer:", "")
  int2 = int1.replace(f"\n","")
  op_list.append(int2)

### final output

In [15]:
ds_test_pd['pred_sentiment'] = op_list
ds_test_pd['pred_sentiment'] = ds_test_pd['pred_sentiment'].map({'Negative':0,'Neutral':1,'Positive':2})
from sklearn.metrics import classification_report
print(classification_report(ds_test_pd['label'], ds_test_pd['pred_sentiment']))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98        76
           1       0.97      0.99      0.98       348
           2       0.99      0.93      0.96       142

    accuracy                           0.98       566
   macro avg       0.98      0.97      0.97       566
weighted avg       0.98      0.98      0.98       566



### Confusion matrix

In [16]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(ds_test_pd['label'], ds_test_pd['pred_sentiment'],labels=[0,1,2])

array([[[488,   2],
        [  1,  75]],

       [[207,  11],
        [  3, 345]],

       [[423,   1],
        [ 10, 132]]])